In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
from bitarray import bitarray
import os
import binascii
import random
import time

In [3]:
max_bins = 10000000
largePrime = 1003162
rand1 = np.random.randint(max_bins)
rand2 = np.random.randint(max_bins)

In [4]:
def tail_count(hashed):
    z=0
    for i in range(-1,-10,-1):
        if hashed[i]== '0':
            z +=1
        else:
                break
    return z

In [5]:
Time= 60*60*5
t_end=time.time()+Time

In [6]:
max_tail = 0
IDs= set()

r = requests.get('http://stream.meetup.com/2/rsvps', stream=True)

for raw_rsvp in r.iter_lines():
    
    zero_count=0
    if raw_rsvp:
        rsvp = json.loads(raw_rsvp)
        c=bin((((rsvp[u'member'][u'member_id'])*rand1+rand2)%largePrime)%max_bins)
        IDs.add(c)
        zero_count=tail_count(c)
        #print(c, zero_count)
        if zero_count> max_tail:
            max_tail= zero_count
        if time.time()>t_end:
            break
    
    

In [7]:
print ('estimate the unique users with 1 hash function is {}'. format(2**max_tail))
print('true number of unique user is {}'.format(len(IDs)))

estimate the unique users with 1 hash function is 512
true number of unique user is 26784


In [8]:
# Adding 5 hash
rand3 = np.random.randint(max_bins)
rand4 = np.random.randint(max_bins)
rand5 = np.random.randint(max_bins)
rand6 = np.random.randint(max_bins)
rand7 = np.random.randint(max_bins)
rand8 = np.random.randint(max_bins)
rand9 = np.random.randint(max_bins)
rand10 = np.random.randint(max_bins)

In [9]:
tail=np.zeros((5,5000))
groups= np.zeros((5,5,10000))
group_max=np.zeros((5,5))
median_tail=[0]*5
IDs=[]
idx=1

In [11]:
r = requests.get('http://stream.meetup.com/2/rsvps', stream=True)

for raw_rsvp in r.iter_lines():
    if idx >=20000:
        break
    zero_count=0
    if raw_rsvp:
        rsvp = json.loads(raw_rsvp)
        if idx < 5000:
            c=bin((((rsvp[u'member'][u'member_id'])*rand1+rand2)%largePrime)%max_bins)
            tail[0,idx]= tail_count(c)
        elif 5000 <= idx < 10000:
            c=bin((((rsvp[u'member'][u'member_id'])*rand3+rand4)%largePrime)%max_bins)
            tail[1,idx - 5000]= tail_count(c)
        elif 10000 <= idx <12000:
            c=bin((((rsvp[u'member'][u'member_id'])*rand5+rand6)%largePrime)%max_bins)
            tail[2,idx - 10000]= tail_count(c)
        elif 12000 <= idx <16000:
            c=bin((((rsvp[u'member'][u'member_id'])*rand7+rand8)%largePrime)%max_bins)
            tail[3,idx - 12000]= tail_count(c)
        else:
            c=bin((((rsvp[u'member'][u'member_id'])*rand9+rand10)%largePrime)%max_bins)
            tail[3,idx - 16000]= tail_count(c)
        
        IDs.append(c)
    idx +=1


In [12]:
groups= np.array_split(np.array(tail),5)
group_max=np.max(groups, axis=2)
median_tail= np.median(group_max, axis=1)
avg=np.mean(median_tail)

In [13]:
print ('estimate the unique users with 5 hash function is {}'. format(int(2**avg)))
print('true number of unique user is {}'.format(len(IDs)))

estimate the unique users with 5 hash function is 12
true number of unique user is 19999
